In [1]:
import openai
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
openai.api_key = ''

In [3]:
def translate_text(value):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Translate the following text to Swedish: '{value}'"},
            ],
        max_tokens=1024,
        temperature=0,
        )
    return response.choices[0]["message"]["content"].strip()

def translate_item(item):
    translated_item = {}
    for key, value in item.items():
        if value:
            translated_value = translate_text(value)
            translated_item[key] = translated_value
        else:
            translated_item[key] = ''
    return translated_item

In [12]:
# Maximum number of parallel requests
MAX_PARALLEL_REQUESTS = 50

# Assuming the input JSON is in a file named 'input.json'
with open('alpaca_data_cleaned.json', 'r') as f:
    data = json.load(f)


In [13]:
def translate(data):
    with ThreadPoolExecutor(max_workers=MAX_PARALLEL_REQUESTS) as executor:
        futures = {executor.submit(translate_item, item): item for item in data}

        for future in tqdm(as_completed(futures), total=len(futures), desc="Translating"):
            translated_data.append(future.result())

    # Save the translated data to a new JSON file named 'translated_data.json'
    with open(f'translated_data_up_to_{start}_to_{end}.json', 'w') as f:
        json.dump(translated_data, f, ensure_ascii=False, indent=4)

    print(f"Translation complete. The translated data is saved in 'translated_data_from_{start}_to_{end}.json'")

In [19]:
for i in range(52000, 53000, 1000):
    start = i
    end = i + 1000
    translated_data = []
    step_data = data[start:end]
    print(start, end)
    translate(step_data)

52000 53000


Translating: 0it [00:00, ?it/s]

Translation complete. The translated data is saved in 'translated_data_from_52000_to_53000.json'
